# IMDB reviews sentiment analyses

This notebook uses Kaggle dataset (https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews).

### Contents

1. [Import dataset](#import_dataset)
1. [Explore data](#explore_data)
1. [Data preparation](#data_preparation)
1. [Create train and test dataset](#train_test_set)
1. [Create a model](#create_model)
1. [Publish the model](#publish_model)
1. [Deploy and score](#deploy_model)

In [ ]:
!pip install nltk

In [ ]:
%matplotlib inline 

import pandas as pd
import numpy as np
import matplotlib as mlp
import matplotlib.pyplot as plt
import seaborn as sns
import json

from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import preprocessing
from sklearn import tree
from sklearn import svm
from sklearn import ensemble
from sklearn import neighbors
from sklearn import linear_model
from sklearn import metrics

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

<a id="import_dataset"></a>
## 1. Import dataset

In [ ]:
# The code was removed by IBM Watson Studio for sharing.

<a id="explore_data"></a>
## 2. Explore data

In [ ]:
df = df_data_1
df.head()

In [ ]:
df.describe()

In [ ]:
ax = sns.countplot(x="sentiment", data=df)
plt.title("Sentiment label distribution")

<a id="data_preparation"></a>
## 3. Data preparation

In this step you will prepare data for training a model. Using the following Text Feature Engineering techniques:

1. Tonkenization         
2. Removes stop words
3. Stemming text (porter)
4. Joining words (tokens) into a single string

In [ ]:
stop_words = stopwords.words('english')
porter_stemmer = PorterStemmer()

In [ ]:
def identify_tokens(row):
    """Identify tokens in a row
    Args:
        row (list): row of dataframe
    
    Returns:
        list: text splited in tokens
    """
    source = row[0]
    tokens = word_tokenize(source)
    token_words = [w for w in tokens if w.isalpha()]
    return token_words

In [ ]:
def remove_stops(row):
    """Remove stop words from text
    Args:
        row (list): row of dataframe
    
    Returns:
        list: list of tokens without stop words
    """
    source_tokenization = row[2]
    stop = [w for w in source_tokenization if not w in stop_words]
    return (stop)

In [ ]:
def stem_porter(row):
    """Execute steamming porter
    Args:
        row (list): row of dataframe
    
    Returns:
        list: list of tokens with steamming.
    """  
    my_list = row[2]
    stemmed_list = [porter_stemmer.stem(word) for word in my_list]
    return (stemmed_list)

In [ ]:
def rejoin_words(row):
    """Join tokens in a single string
    Args:
        row (list): row of dataframe
    
    Returns:
        str: text of joined tokens
    """   
    my_list = row[2]
    joined_words = (" ".join(my_list))
    return joined_words

In [ ]:
def pre_processing(df):
    """Execute text feature engineering (TFE)
    Args:
        df (dataframe): row of dataframe
    
    Returns:
        list: Text post text feature engineering (TFE)
    """  
    print('Tokenization ...')
    df['text1'] = df.apply(identify_tokens, axis=1)
    print('Removing stop words ...')
    df['text1'] = df.apply(remove_stops, axis=1)
    print('Stemming (porter) ...')
    df['text1'] = df.apply(stem_porter, axis=1)
    print('Joining words ...')
    df['clean_text'] = df.apply(rejoin_words, axis=1)
    print('DONE!')
    
    return df

In [ ]:
df = pre_processing(df)

df['clean_text'] = df['clean_text'].str.lower()
df.head()

<a id="train_test_set"></a>
## 4. Create train and test dataset

NOTE: Test dataset (30%) and Training dataset (70%) balanced (Stratified)

In [ ]:
X = df['clean_text']
Y = df['sentiment']

print(X.shape)
print(Y.shape)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, stratify=Y)

NOTE: Machine Learning or Deep Learning models uses numeric values input. The Tf-Idf Text Feature Engineering (TFE) process will be used to transform the texts into vectors.

In [ ]:
tfidf = TfidfVectorizer(max_features=2000, ngram_range=(2,3), sublinear_tf=True)

X_train_tf = tfidf.fit_transform(X_train)
X_test_tf = tfidf.transform(X_test)

print(Y.value_counts().shape)
print(X_train_tf.shape)

In [ ]:
le = preprocessing.LabelEncoder()

Y_train_le = le.fit_transform(list(Y_train))
Y_test_le = le.transform(list(Y_test))

<a id="create_model"></a>
## 5. Create a model

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

In [ ]:
# Binary classifiers
# GradientBoostingClassifier
gradient_boost = GradientBoostingClassifier()
gradient_boost.fit(X_train_tf, Y_train_le)
Y_predict_gradient_boost = gradient_boost.predict(X_test_tf)
print('Gradient Boosting Classifier DONE!')

# SVC -- taking too much time
# svc_model = SVC(gamma='auto', kernel='sigmoid', C=1.8, probability=True)
# svc_model.fit(X_train_tf, Y_train_le)
# Y_predict_svm = svc_model.predict(X_test_tf)
# print('Support Vector Machine(SVM) DONE!')

# RandomForestClassifier
random_forest = RandomForestClassifier(n_estimators=10)
random_forest.fit(X_train_tf, Y_train_le)
Y_predict_random_forest = random_forest.predict(X_test_tf)
print('Random Forest Classifier DONE!')

# KNeighborsClassifier
k_neighbors = KNeighborsClassifier()
k_neighbors.fit(X_train_tf, Y_train_le)
Y_predict_k_neighbors = k_neighbors.predict(X_test_tf)
print('K Nearest Neighbor Classifier DONE!')

# LogisticRegression
logistic_regression = LogisticRegression(solver='lbfgs', penalty='l2', C=1.5)
logistic_regression.fit(X_train_tf, Y_train_le)
Y_predict_logistic_regression = logistic_regression.predict(X_test_tf)
print('Logistic Regression DONE!')

### 5.1 Model evaluation

In [ ]:
print('Gradient Boosting Classifier:  ', metrics.accuracy_score(Y_test_le, Y_predict_gradient_boost))
# print('Support Vector Machine(SVM):   ', metrics.accuracy_score(Y_test_le, Y_predict_svm))
print('Random Forest Classifier:      ', metrics.accuracy_score(Y_test_le, Y_predict_random_forest))
print('K Nearest Neighbor Classifier: ', metrics.accuracy_score(Y_test_le, Y_predict_k_neighbors))
print('Logistic Regression:           ', metrics.accuracy_score(Y_test_le, Y_predict_logistic_regression))

### 5.2 Resume of classification

NOTE: Accuracy >= 0.70

In [ ]:
# print('Support vector machine(SVM):\n {}\n'.format(metrics.classification_report(Y_test_le, Y_predict_svm)))
print('Random Forest Classifier:\n {}\n'.format(metrics.classification_report(Y_test_le, Y_predict_random_forest)))
print('Logistic Regression:\n {}\n'.format(metrics.classification_report(Y_test_le, Y_predict_logistic_regression)))

### 5.3 - Logistic Regression

NOTE: Best accuracy model confusion matrix

In [ ]:
logistic_regression_conf_matrix = metrics.confusion_matrix(Y_test_le, Y_predict_logistic_regression)
sns.heatmap(logistic_regression_conf_matrix, annot=True,  fmt='');
plt.title('Confusion matrix, Logistic Regression');

NOTE: Best accuracy model ROC curve

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(Y_test_le, Y_predict_logistic_regression)
auc = metrics.roc_auc_score(Y_test_le, Y_predict_logistic_regression)

plt.subplots(figsize=(5,5))
plt.plot(fpr,tpr, label="AUC="+str(auc))
plt.plot(np.linspace(0, 1, 100),
         np.linspace(0, 1, 100),
         label='baseline',
         linestyle='--')
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve', fontsize=18)
plt.legend(loc=4)
plt.show()

### 5.4 Model selection

In [ ]:
X_train_final = tfidf.fit_transform(X)
Y_train_final = le.fit_transform(list(Y))

print(X_train_final.shape)

In [ ]:
lrc = LogisticRegression(solver='lbfgs', penalty='l2', C=1.5)
lrc.fit(X_train_final, Y_train_final)

<a id="publish_model"></a>
## 6. Publish the model

To authenticate to Watson Machine Learning in the IBM Cloud, you need api_key and service location.

Using [IBM Cloud CLI](https://cloud.ibm.com/docs/cli/index.html) or directly through the IBM Cloud portal.

Using IBM Cloud CLI:

```
ibmcloud login
ibmcloud iam api-key-create API_KEY_NAME
```

NOTE: To get the Service URL [Endpoint URLs section of the Watson Machine Learning docs](https://cloud.ibm.com/apidocs/machine-learning).

In [ ]:
api_key = 'API_KEY'
location = 'LOCATION'

In [ ]:
wml_credentials = {
    "apikey": api_key,
    "url": location
}

### 6.1 Installing IBM Watson Machine Learning library

NOTE: Documentation could be found [here](http://ibm-wml-api-pyclient.mybluemix.net/)

In [ ]:
!pip install -U ibm-watson-machine-learning

In [ ]:
from ibm_watson_machine_learning import APIClient

client = APIClient(wml_credentials)
print(client.version)

### 6.2 Publish model to project

In [ ]:
project_id = 'PROJECT_ID'

In [ ]:
client.set.default_project(project_id)

In [ ]:
sofware_spec_uid = client.software_specifications.get_id_by_name("runtime-22.1-py3.9")
metadata = {
            client.repository.ModelMetaNames.NAME: 'Logistic Regression model to predict IMDB reviews',
            client.repository.ModelMetaNames.TYPE: 'scikit-learn_1.0',
            client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: sofware_spec_uid
}

published_model = client.repository.store_model(model=lrc, meta_props=metadata)

### 6.3 Publish model to deployment space

In [ ]:
space_id = 'SPACE_ID'

In [ ]:
client.set.default_space(space_id)

In [ ]:
client.spaces.list(limit=10)

In [ ]:
published_model = client.repository.store_model(model=lrc, meta_props=metadata)

In [ ]:
published_model_uid = client.repository.get_model_id(published_model)
model_details = client.repository.get_details(published_model_uid)
print(json.dumps(model_details, indent=2))

In [ ]:
client.repository.list_models()

In [ ]:
# client.repository.delete('MODEL_ID')

<a id="deploy_model"></a>
## 7. Deploy and Score

NOTE: Deploy and score the model deployed at IBM Watson Machine Learning

In [ ]:
metadata = {
    client.deployments.ConfigurationMetaNames.NAME: "Deployment of IMDB reviews model",
    client.deployments.ConfigurationMetaNames.ONLINE: {}
}

created_deployment = client.deployments.create(published_model_uid, meta_props=metadata)

In [ ]:
# Get deployment UID and show details on the deployment
deployment_uid = client.deployments.get_uid(created_deployment)
client.deployments.get_details(deployment_uid)

In [ ]:
client.deployments.list()

In [ ]:
# client.deployments.delete('GUID of deployed model')

### 7.1 Score model

NOTE: Test the API created from IBM Watson Machine Learning.

In [ ]:
# get scoring end point
scoring_endpoint = client.deployments.get_scoring_href(created_deployment)
print(scoring_endpoint)

In [ ]:
# add some test data
scoring_payload = {"input_data": [
    {'values': X_test_tf.toarray()
    }]}

In [ ]:
# score the model
predictions = client.deployments.score(deployment_uid, scoring_payload)
print('prediction',json.dumps(predictions, indent=2))

In [ ]:
Y_predict_final_model = []
for y in predictions['predictions'][0]['values']:
    Y_predict_final_model.append(y[0])
    
print('Final Model WML:\n {}\n'.format(metrics.classification_report(Y_test_le, Y_predict_final_model)))